In [26]:
# we load the things!

from ngsolve import *
#import netgen.gui
from netgen.csg import * 
from netgen.occ import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.solvers import GMRes

import scipy.sparse as sp
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psutil

In [27]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of B vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of B scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed B scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of B vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

In [28]:
def generateGeometry(hmax):

    largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))
    smallBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0, 0, 0))

    reentrantCornerGeo3D = largeBrick - smallBrick
    # Here instead of the following mesh command, I think I need to implement a structured mesh with the manual mesh generation approach.
    mesh = Mesh(OCCGeometry(reentrantCornerGeo3D).GenerateMesh(maxh=hmax))

    return mesh

In [29]:

GMRes = True # If GMRes is set to false, PARDISO solver is used, otherwise, GMRes.
   
u_m = CF(( -cos(2*x)*sin(2*y)*cos(2*z), 
            sin(2*x)*cos(2*y)*sin(2*z),
            -cos(2*x)*sin(2*y)*cos(2*z)))
C_w = 10
order = 1

mesh = generateGeometry(0.5)
#mesh.Refine()

In [33]:
h_curl = HCurl(mesh, order=order, type1=False)  # For 1-forms, H(curl)
h_1 = H1(mesh, order=order+1)     # For 0-forms, H1 space
fes = h_curl * h_1
(u, p), (v, q) = fes.TnT()

B = BilinearForm(fes)

B += u * grad(q) * dx
B += - p * q * dx

B +=  grad(p) * v * dx
B +=  curl(u) * curl(v) * dx

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

B += - curl(u) * Cross(n, v) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
B += - Cross(n, u) * curl(v) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
B += (C_w / h) * Cross(n, u) * Cross(n, v) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

f = CF(GCurl(GCurl(u_m)) - GGrad(GDiv(u_m))) 

rhs = LinearForm(fes)
rhs += f * v * dx

rhs += (C_w / h) * Cross(n, u_m) * Cross(n, v) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
rhs += - Cross(n, u_m) * curl(v) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))
rhs += u_m * n * q * ds(skeleton=True, definedon=mesh.Boundaries(".*"))


with TaskManager():

    if (GMRes == False):
        B.Assemble()
        rhs.Assemble()
        sol = GridFunction(fes)

        res = rhs.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
        
        sol.vec.data += inv * res

    else:
        #myprec = Preconditioner(B, type="multigrid")
        B.Assemble()
        rhs.Assemble()
        
        sol = GridFunction(fes)
        res = rhs.vec - B.mat * sol.vec

        # rows,cols,vals = B.mat.COO()
        # A = sp.csr_matrix((vals,(rows,cols)))
        # A_diag = A.diagonal()
        # inv_diag = np.where(np.abs(A_diag) > 1e-15, 1.0/A_diag, 0.0)
        # A_jacobi = sp.diags(inv_diag, offsets=0, shape=A.shape, format='coo')

        # #myprec = B.mat.CreateSmoother(fes.FreeDofs())
        # myprec = IdentityMatrix(fes.ndof, complex=False)
        # BtB = B.mat @ B.mat.CreateTranspose
        # lams = EigenValues_Preconditioner(mat=BtB, pre=myprec)
        # singular_values = np.sqrt(lams)  # Take square root of eigenvalues to get singular values
        # print("Condition Number Kappa: ", max(singular_values)/min(singular_values))


        # #inv = CGSolver(B.mat, myprec.mat, maxsteps=1000)
        # #print(type(fes.FreeDofs()))

        # inv =   GMRESSolver(mat         = B.mat,       
        #                     pre        = myprec,
        #                     #freedofs    = fes.FreeDofs(),        
        #                     printrates  = False,         
        #                     precision   = 1e-8,
        #                     maxsteps    = 1000)

        preId = IdentityMatrix()
        preId = Projector(fes.FreeDofs(), True)
        print(type(fes.FreeDofs()))
        inv = GMRes(B.mat, rhs.vec, freedofs=fes.FreeDofs(), printrates="\r", restart=50)

        sol.vec.data += inv * res

    cond_nr = np.linalg.cond(A.todense())
    eigenvalues = np.linalg.eigvals(A.todense())

    print("Condition Number Kappa: ", cond_nr)
    print("Eigenvalues:", eigenvalues)

    #plt.plot(eigenvalues)
    
    gf_u , gf_p = sol.components

<class 'pyngcore.pyngcore.BitArray'>


TypeError: 'bool' object is not callable

In [72]:
curl_u = curl(gf_u)
grad_p = grad(gf_p)

curl_u_m = CF(GCurl(u_m))
p_m = - CF(GDiv(u_m))
grad_p_m = CF(GGrad(p_m))

gf_u_n = CF(gf_u * n)
gf_u_n_m = CF(u_m * n)

gf_gamma_p_u = CF(Cross(n, gf_u))
gf_gamma_p_u_m = CF(Cross(n, u_m))

L2_error_u = sqrt(Integrate((gf_u - u_m)**2, mesh))
L2_error_curl_u = sqrt(Integrate((curl_u - curl_u_m)**2, mesh))
L2_error_p = sqrt(Integrate((gf_p - p_m)**2, mesh))
L2_error_grad_p = sqrt(Integrate((grad_p - grad_p_m)**2, mesh))

L2_error_gamma_p_u = sqrt(Integrate((gf_gamma_p_u - gf_gamma_p_u_m)**2, mesh, BND))
L2_error_u_n = sqrt(Integrate((gf_u_n - gf_u_n_m)**2, mesh, BND))
L2_error_p_Gamma = sqrt(Integrate((gf_p - p_m)**2, mesh, BND))